# ⚖️ Scales

Scales, provided by [`d3-scale-chromatic`](https://github.com/d3/d3-scale-chromatic)
provide an efficient way to use pre-calcluated values for a number color-based node and
link [shape](./Shapes.ipynb) properties.

Scales have a _domain_ (the expected values) and a _range_ (the colors to draw).

Some experimentation may be needed to find an appropriate combination of these settings
for a given graph's data.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import ipywidgets as W
import traitlets as T
from ipyforcegraph import behaviors as B
from ipyforcegraph import graphs as G
from ipyforcegraph.behaviors import scales as SC

In [ ]:
fg = G.ForceGraph(layout=dict(flex="1", height="100%"))
controls = W.VBox(layout=dict(min_width="25em", flex="0"))
W.HBox(
    [controls, fg],
    layout=dict(
        min_height="400px", height="100%", flex="1", overflow="hidden", display="flex"
    ),
)

In [ ]:
n = 100
with fg.source.hold_trait_notifications():
    fg.source.nodes = [{"id": i, "value": i / n} for i in range(n)]
    fg.source.links = sum(
        [
            [
                {"id": i, "source": i, "target": i - 1 if i else n - 1},
                {"id": i, "source": i, "target": i % 5, "group": i % 7},
            ]
            for i in range(n)
        ],
        [],
    )

In [ ]:
ns = B.NodeShapes()
ls = B.LinkShapes()
fg.behaviors = [ns, ls]

In [ ]:
def make_a_scheme_picker(enum):
    return W.SelectionSlider(
        description="scheme", options={v.name: v.value for v in enum}
    )

In [ ]:
def make_a_domain_range(scale):
    table = W.VBox()

    def update_scale(*_):
        scale.domain = [row.children[1].value for row in table.children]
        scale.range = [row.children[2].value for row in table.children]

    def remove_row(row):
        table.children = [c for c in table.children if c != row]
        row.close()

    def add_row(*_):
        btn_remove_row = W.Button(
            icon="trash", button_style="danger", layout=dict(flex="0", max_width="3em")
        )
        d = W.FloatText(layout=dict(flex="1", max_width="4em"))
        r = W.ColorPicker(layout=dict(flex="1", width="unset"))
        d.observe(update_scale)
        r.observe(update_scale)
        row = W.HBox([btn_remove_row, d, r])
        btn_remove_row.on_click(lambda *_: remove_row(row))
        table.children = [*table.children, row]
        update_scale()

    btn_add_row = W.Button(icon="plus", description="add row")
    btn_add_row.on_click(add_row)
    ui = W.VBox([btn_add_row, table])
    return ui

In [ ]:
def make_a_scale_picker(behavior, column, domain):
    c_scale = SC.ContinuousColor(
        column, scheme=SC.ContinuousColor.Scheme.viridis, domain=domain
    )
    c_scheme = make_a_scheme_picker(SC.ContinuousColor.Scheme)
    c_ui = W.VBox([c_scheme])
    T.link((c_scale, "scheme"), (c_scheme, "value"))

    o_scale = SC.OrdinalColor(
        column, scheme=SC.OrdinalColor.Scheme.accent, domain=domain
    )
    o_scheme = make_a_scheme_picker(SC.OrdinalColor.Scheme)
    o_ui = W.VBox([o_scheme])
    T.link((o_scale, "scheme"), (o_scheme, "value"))

    m_scale = SC.OrdinalColor(column)
    m_domain_range = make_a_domain_range(m_scale)
    m_ui = W.VBox([m_domain_range])

    tabs = W.Tab([c_ui, o_ui, m_ui], titles=["continuous", "ordinal", "manual"])

    T.dlink(
        (tabs, "selected_index"),
        (behavior, "color"),
        lambda i: [c_scale, o_scale, m_scale][i],
    )
    return tabs

In [ ]:
controls.children = [
    W.Label("nodes"),
    make_a_scale_picker(ns, "value", (0.0, 1.0)),
    W.Label("links"),
    make_a_scale_picker(ls, "group", (0, 7)),
]